# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid, generate_time_series
import time

In [2]:
# summary
summary_df = pd.DataFrame(
    np.zeros(shape=(8, 3), dtype=np.float64), 
    columns=['PGM Linear', 'PGM NR', 'PandaPower NR'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = n_node_per_feeder * n_feeder

# Pre-cache Library


In [4]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Single Calculation

In [5]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']


## Symmetric

In [6]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [7]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM NR'] = end - start


In [8]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower NR'] = end - start

In [9]:
np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()

2.3988731663537877e-05

In [10]:
np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

0.0002488719126692851

## Asymmetric

In [11]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [12]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM NR'] = end - start

In [13]:
pp_net.load[['p_mw', 'q_mvar']] = 0.0


# first calculation with solver initialization
start = time.time()
pp.pf.runpp_3ph.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.pf.runpp_3ph.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower NR'] = end - start


In [14]:
np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()

5.79704284575655e-10

In [15]:
np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

8.192209383084403e-09

# Time Series Calculation

In [16]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']

## Symmetric

In [17]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM Linear'] = end - start

In [18]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM NR'] = end - start

## Asymmetric

In [19]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM Linear'] = end - start

In [20]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM NR'] = end - start

# Summary of Performace

In [21]:
summary_df

,PGM Linear,PGM NR,PandaPower NR
Symmetric calculation with solver initialization,0.002999,0.005981,0.023997
Symmetric calculation without solver initialization,0.000999,0.005005,0.017995
Asymmetric calculation with solver initialization,0.004995,0.025024,0.142042
Asymmetric calculation without solver initialization,0.004040,0.018000,0.130038
Time series symmetric calculation,1.005065,4.160848,0.000000
Time series asymmetric calculation,2.372059,16.754472,0.000000
N-1 symmetric calculation,0.000000,0.000000,0.000000
N-1 asymmetric calculation,0.000000,0.000000,0.000000
